In [1]:
# ライブラリをインポート

# データ操作と数値計算のため
import pandas as pd
import numpy as np

# データ可視化のため
import matplotlib.pyplot as plt
import seaborn as sns

# 統計モデリングと計量経済分析のため
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# for ignore warning
import warnings
warnings.filterwarnings('ignore')

In [3]:
url = "https://www.fbc.keio.ac.jp/~tyabu/keiryo/money_data.csv"
df = pd.read_csv(url)

df['lnm'] = np.log(df['M1'] / df['NGDP'])
df['lnr'] = np.log(df['interest'])

df.head()

,M1,NGDP,interest,lnm,lnr
0,388.100000,2796.523,0.133533,-1.974869,-2.013404
1,385.900000,2799.938,0.096167,-1.981774,-2.341672
2,399.333333,2860.044,0.091533,-1.968796,-2.391052
3,409.400000,2993.525,0.136133,-1.989514,-1.994120
4,415.033333,3131.799,0.143900,-2.021004,-1.938637


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 136 entries, 1980-03-31 to 2013-12-31
Freq: QE-DEC
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   M1        136 non-null    float64
 1   NGDP      136 non-null    float64
 2   interest  136 non-null    float64
 3   lnm       136 non-null    float64
 4   lnr       136 non-null    float64
dtypes: float64(5)
memory usage: 6.4 KB


In [5]:
df.describe()

,M1,NGDP,interest,lnm,lnr
count,136.000000,136.000000,136.000000,136.000000,136.000000
mean,1427.987745,9040.619529,0.048235,-1.887515,-3.662694
std,802.840900,4286.796765,0.035185,0.097812,1.603339
min,385.900000,2796.523000,0.000133,-2.033224,-8.922661
25%,775.108333,5276.541250,0.018600,-1.969434,-3.985572
50%,1254.566667,8344.569000,0.049183,-1.893623,-3.012203
75%,2044.016666,13031.923500,0.069200,-1.848839,-2.670771
max,3436.500000,16999.888000,0.150533,-1.598753,-1.893571


# 四半期データ(1980Q1-2013Q4)
※変数の定義をしてから時系列データとして読み込みをする

In [16]:
# 日付インデックスを作成
date_index = pd.date_range(
    start = '1980-01-01',
    periods = len(df),
    freq = 'Q'                 # 四半期
)

# データフレームにインデックスを設定
df.index = date_index

# 結果を確認
df.head()

,M1,NGDP,interest,lnm,lnr
1980-03-31,388.100000,2796.523,0.133533,-1.974869,-2.013404
1980-06-30,385.900000,2799.938,0.096167,-1.981774,-2.341672
1980-09-30,399.333333,2860.044,0.091533,-1.968796,-2.391052
1980-12-31,409.400000,2993.525,0.136133,-1.989514,-1.994120
1981-03-31,415.033333,3131.799,0.143900,-2.021004,-1.938637


# 6.3.2節の推定

In [20]:
# 対数対数モデル
endog = df['lnm']
exog = df['lnr']
exog = sm.add_constant(exog)

mod_1 = sm.OLS(
    endog,
    exog
)
res_1 = mod_1.fit()
print(res_1.summary())

                            OLS Regression Results                            
Dep. Variable:                    lnm   R-squared:                       0.816
Model:                            OLS   Adj. R-squared:                  0.814
Method:                 Least Squares   F-statistic:                     593.2
Date:                Mon, 29 Jul 2024   Prob (F-statistic):           4.63e-51
Time:                        12:35:48   Log-Likelihood:                 238.70
No. Observations:                 136   AIC:                            -473.4
Df Residuals:                     134   BIC:                            -467.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0893      0.009   -231.130      0.0

In [23]:
# 対数線形モデル
endog = df['lnm']
exog = df['interest']
exog = sm.add_constant(exog)

mod_2 = sm.OLS(
    endog,
    exog
)
res_2 = mod_2.fit()
print(res_2.summary())

                            OLS Regression Results                            
Dep. Variable:                    lnm   R-squared:                       0.671
Model:                            OLS   Adj. R-squared:                  0.668
Method:                 Least Squares   F-statistic:                     272.7
Date:                Mon, 29 Jul 2024   Prob (F-statistic):           4.15e-34
Time:                        12:37:34   Log-Likelihood:                 199.18
No. Observations:                 136   AIC:                            -394.4
Df Residuals:                     134   BIC:                            -388.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.7777      0.008   -216.277      0.0

In [22]:
# 推定結果を1つの表にまとめる．

results_table = summary_col(
    results = [
        res_1,
        res_2
    ],
    model_names = [
        'Eq(1)',
        'Eq(2)'
    ],
    stars = True,
    float_format = '%0.3f',
    info_dict = {
        'N':lambda x: "{0:d}".format(int(x.nobs))
    }
)

print(results_table)


                 Eq(1)     Eq(2)  
----------------------------------
const          -2.089*** -1.778***
               (0.009)   (0.008)  
lnr            -0.055***          
               (0.002)            
interest                 -2.276***
                         (0.138)  
R-squared      0.816     0.671    
R-squared Adj. 0.814     0.668    
N              136       136      
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01
